In [1]:
import os
import tensorflow as tf
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split


In [2]:
DATASET_PATH = './datasets/'
CATEGORIES = ['negatives', 'positives']
IMG_SIZE = 128

In [3]:
def read_data():
    X = []
    y = []
    
    for category in CATEGORIES:
        path = os.path.join(DATASET_PATH, category)
        label = CATEGORIES.index(category)

        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            features = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(4, 4), visualize=False)
            X.append(features.tolist())
            y.append(label)
        
    return np.array(X), np.array(y)

In [4]:
X, y = read_data()

In [5]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
(X_train, X_valid, y_train, y_valid) = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation ="relu"),
    tf.keras.layers.Dense(128, activation ="relu"),
    tf.keras.layers.Dense(128, activation ="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer="rmsprop", loss="BinaryCrossentropy", metrics=["accuracy"])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-17 17:48:49.046598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-17 17:48:49.046743: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


2022-12-17 17:48:49.242841: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-17 17:48:49.521922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


264/264 [==============================] - ETA: 0s - loss: 0.3583 - accuracy: 0.8434

2022-12-17 17:48:52.520650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


264/264 [==============================] - 4s 12ms/step - loss: 0.3583 - accuracy: 0.8434 - val_loss: 0.4592 - val_accuracy: 0.7945
Epoch 2/20
264/264 [==============================] - 3s 11ms/step - loss: 0.2691 - accuracy: 0.8913 - val_loss: 0.2804 - val_accuracy: 0.8828
Epoch 3/20
264/264 [==============================] - 3s 10ms/step - loss: 0.2444 - accuracy: 0.9033 - val_loss: 0.2605 - val_accuracy: 0.8951
Epoch 4/20
264/264 [==============================] - 3s 10ms/step - loss: 0.2243 - accuracy: 0.9105 - val_loss: 0.2611 - val_accuracy: 0.8932
Epoch 5/20
264/264 [==============================] - 3s 10ms/step - loss: 0.2085 - accuracy: 0.9128 - val_loss: 0.2589 - val_accuracy: 0.8975
Epoch 6/20
264/264 [==============================] - 3s 10ms/step - loss: 0.1930 - accuracy: 0.9204 - val_loss: 0.3226 - val_accuracy: 0.8752
Epoch 7/20
264/264 [==============================] - 3s 10ms/step - loss: 0.1872 - accuracy: 0.9228 - val_loss: 0.2628 - val_accuracy: 0.9041
Epoch 8/20

In [10]:
model.evaluate(X_test, y_test)

83/83 [==============================] - 0s 4ms/step - loss: 0.6481 - accuracy: 0.9157


[0.6481258273124695, 0.9156855344772339]

In [11]:
tf.keras.models.save_model(model, "model")

2022-12-17 17:52:02.681334: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/assets
